In [1]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec

In [2]:
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
name="rag" , dimension=1536 , metric="cosine" , spec=ServerlessSpec(cloud="aws" , region="us-east-1")
)

PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '8b9e50cd05988de9fe1521c45171c4b3', 'Date': 'Sat, 24 Aug 2024 14:10:55 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [3]:
 import json 
 data = json.load(open("reviews.json"))
 data['reviews']

[{'professor': 'Dr. Emily Carter',
  'subject': 'Introduction to Computer Science',
  'stars': 5,
  'review': 'Dr. Carter is an amazing professor! Her lectures are clear, and she always makes time for questions.'},
 {'professor': 'Dr. John Stevens',
  'subject': 'Calculus I',
  'stars': 4,
  'review': 'Great explanations, but the pace is a bit fast. Overall, a very knowledgeable professor.'},
 {'professor': 'Prof. Susan Reynolds',
  'subject': 'Modern Art History',
  'stars': 3,
  'review': 'Interesting lectures, but the grading is quite tough.'},
 {'professor': 'Dr. Alan Thompson',
  'subject': 'Organic Chemistry',
  'stars': 2,
  'review': "Content is challenging, but Dr. Thompson's teaching style is hard to follow."},
 {'professor': 'Dr. Jane Matthews',
  'subject': 'Psychology 101',
  'stars': 5,
  'review': 'Dr. Matthews is engaging and makes the material easy to understand. Highly recommend!'},
 {'professor': 'Prof. Michael Lee',
  'subject': 'World History',
  'stars': 4,
  'rev

In [4]:
 processed_data = []
 client = OpenAI()
 for review in data['reviews']:
     response = client.embeddings.create(
         input=review['review'],
         model="text-embedding-3-small"
     )
     embedding = response.data[0].embedding
     processed_data.append({
         "values": embedding,
         "id" : review["professor"],
         "metadata": {
             "review" :review["review"],
             "subject":review["subject"],
             "stars" :review["stars"]
         }
     })

In [5]:
 index=pc.Index('rag')
 index.upsert(
     vectors=processed_data,
     namespace="ns1"
)

{'upserted_count': 20}

In [6]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 20}},
 'total_vector_count': 20}